# AutoML на текстовых данных

![NLP](./imgs/tutorial_NLP_image_1.jpg)


Чуть больше про стратегии получения представлений текстов на основе представлений слов:

![NLP2](./imgs/tutorial_NLP_image_2.jpg)

Про методы случайных алгоритмов можно подробнее прочитать в [статье](https://arxiv.org/abs/1901.10444) "No Training Required: Exploring Random Encoders for Sentence Classification".


# Импорты

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
from lightautoml.addons.interpretation import LimeTextExplainer
from lightautoml.report import ReportDecoNLP

# Выключим предупреждения от HuggingFace
import transformers
transformers.logging.set_verbosity(50)

# Чтение данных

In [5]:
%%time
train = pd.read_csv("example_data/nlp_data.csv")

CPU times: user 230 ms, sys: 9.2 ms, total: 239 ms
Wall time: 256 ms


In [6]:
print(train.shape)
train.sample(5, random_state=0)

(13842, 6)


,BankName,Message,ViewsNum,IsGood,MessageRecognized,WER
11474,Альфа-Банк,"Я клиент банка с 2007 года, зарплатный клиент ...",1422,False,Я клиент банка с две тысячи седьмого года зарп...,60.000000
3955,Альфа-Банк,07.04 в 20-15 по Ульяновскому времени я зашла ...,2016,False,Седьмого апреля в двадцать пятнадцать По Ульян...,31.818182
3081,Банк Открытие,Ужасный сервис. Заказал кредитную карту по акц...,2232,False,Ужасной Сервис заказал кредитную карту по акци...,68.750000
12107,Почта Банк,"Добрый вечер. 21.01.2020, я обратилась в отде...",1139,False,Добрый вечер двадцать первого января две тысяч...,55.555556
10494,Русский Стандарт,Банк второй месяц подряд еженедельно названива...,1609,False,Второй месяц подряд еженедельно и предлагает к...,39.393939


In [8]:
train.WER.mean()

45.355170973294456

In [10]:
train, test = train_test_split(train, test_size=3_000, random_state=42, stratify=train.IsGood)

## На чистой речи

In [13]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'Message'],
         'drop': ['MessageRecognized', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['nn']},
                          nn_params = {'lang': 'ru', 'bert_name': "cointegrated/rubert-tiny"},
                          )

oof_pred = automl.fit_predict(train, roles=roles, verbose=3) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

[18:43:20] Stdout logging level is INFO3.
[18:43:20] Model language mode: ru
[18:43:20] Task: binary

[18:43:20] Start automl preset with listed constraints:
[18:43:20] - time: 3600.00 seconds
[18:43:20] - CPU: 4 cores
[18:43:20] - memory: 16 GB

[18:43:20] Train data shape: (10842, 6)

[18:43:20] Feats was rejected during automatic roles guess: []
[18:43:20] Layer 1 train process start. Time left 3599.94 secs
[18:43:20] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[18:43:20] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 226/226 [00:04<00:00, 55.67it/s]


[18:43:49] Epoch: 0, train loss: 0.4959859848022461, val loss: 0.47765660285949707, val metric: 0.7716027986391938


val: 100%|██████████| 226/226 [00:03<00:00, 57.80it/s]


[18:44:09] Epoch: 1, train loss: 0.44830235838890076, val loss: 0.43643373250961304, val metric: 0.8280139694222755


...

[18:50:02] Epoch: 4, train loss: 0.3364371657371521, val loss: 0.36153361201286316, val metric: 0.8565286987251344


val: 100%|██████████| 226/226 [00:04<00:00, 55.74it/s]


[18:50:23] Epoch: 5, train loss: 0.3131820559501648, val loss: 0.36288097500801086, val metric: 0.858519132483423


val: 100%|██████████| 226/226 [00:03<00:00, 56.67it/s]


[18:50:27] Early stopping: val loss: 0.36153361201286316, val metric: 0.8565286987251344
[18:50:27] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN finished. score = 0.8302490019515232
[18:50:27] Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
[18:50:27] Time left 3173.22 secs

[18:50:27] Layer 1 training completed.

[18:50:27] Automl preset training completed in 426.79 seconds

[18:50:27] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (3 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN) 



In [14]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8302490019515232


In [15]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

test: 100%|██████████| 188/188 [00:03<00:00, 53.48it/s]


AUC TEST score: 0.853947299491854
CPU times: user 8.73 s, sys: 3.07 s, total: 11.8 s
Wall time: 29.5 s


In [16]:
print('F1 TEST score: {}'.format(f1_score(test[roles['target']].values, test_pred.data[:, 0] > 0.5)))

F1 TEST score: 0.5964912280701753


## На распознанной речи

In [17]:
roles = {'target': 'IsGood',
         'text': ['BankName', 'MessageRecognized'],
         'drop': ['Message', 'WER']}

task = Task('binary')

automl = TabularNLPAutoML(task = task, 
                          timeout = 3600,
                          gpu_ids = '1',
                          general_params = {'use_algos': ['nn']},
                          nn_params = {'lang': 'ru', 'bert_name': "cointegrated/rubert-tiny"},
                          )

oof_pred = automl.fit_predict(train, roles=roles, verbose=3) 
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

[18:51:57] Stdout logging level is INFO3.
[18:51:57] Model language mode: ru
[18:51:57] Task: binary

[18:51:57] Start automl preset with listed constraints:
[18:51:57] - time: 3600.00 seconds
[18:51:57] - CPU: 4 cores
[18:51:57] - memory: 16 GB

[18:51:57] Train data shape: (10842, 6)

[18:51:57] Feats was rejected during automatic roles guess: []
[18:51:57] Layer 1 train process start. Time left 3599.94 secs
[18:51:57] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[18:51:57] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 226/226 [00:03<00:00, 56.66it/s]


[18:52:24] Epoch: 0, train loss: 0.49870461225509644, val loss: 0.4847054183483124, val metric: 0.747906369685688


val: 100%|██████████| 226/226 [00:03<00:00, 57.01it/s]


[18:52:45] Epoch: 1, train loss: 0.46062275767326355, val loss: 0.4456520974636078, val metric: 0.8081030256779663


...

[18:58:12] Epoch: 2, train loss: 0.4159608781337738, val loss: 0.40527674555778503, val metric: 0.8255051488609808


val: 100%|██████████| 226/226 [00:04<00:00, 55.29it/s]


[18:58:32] Epoch: 3, train loss: 0.38174155354499817, val loss: 0.40635257959365845, val metric: 0.8299441415081792


val: 100%|██████████| 226/226 [00:04<00:00, 56.46it/s]


[18:58:36] Early stopping: val loss: 0.40527674555778503, val metric: 0.8255051488609808
[18:58:37] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN finished. score = 0.8064579197243278
[18:58:37] Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
[18:58:37] Time left 3200.29 secs

[18:58:37] Layer 1 training completed.

[18:58:37] Automl preset training completed in 399.71 seconds

[18:58:37] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (3 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN) 



In [18]:
print('AUC OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))

AUC OOF score: 0.8064579197243278


In [19]:
%%time 

test_pred = automl.predict(test)
print('AUC TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

test: 100%|██████████| 188/188 [00:03<00:00, 55.57it/s]


AUC TEST score: 0.8327002186434183
CPU times: user 8.88 s, sys: 3 s, total: 11.9 s
Wall time: 29.1 s


In [20]:
print('F1 TEST score: {}'.format(f1_score(test[roles['target']].values, test_pred.data[:, 0] > 0.5)))

F1 TEST score: 0.5728250244379277
